In [20]:
%matplotlib inline
#load packages and data
import datetime
import holidays
import re #regular expressions
import MySQLdb
import pandas.io.sql as sql
import pandas as pd
import numpy as np
import scipy.stats
from matplotlib import pyplot as plt
pd.set_option('display.max_rows', 2000)
import seaborn as sns

In [21]:
#get data from db
conn = MySQLdb.connect(host="localhost", port=3306, user="root", db="disney_db") #make db connection
cursor = conn.cursor()

In [22]:
#import twitter data
cursor.execute("SELECT id, timestamp, location FROM twitter_data_copy WHERE location = 'anaheim' AND timestamp > '2015-08-04' AND timestamp < '2015-09-12' ")
rows = cursor.fetchall()
tweetdf = pd.DataFrame( [[ij for ij in i] for i in rows] )
tweetdf.rename(columns={0: 'tweetid', 1: 'timestamps'}, inplace=True)
print tweetdf.head()

   tweetid          timestamps        2
0      722 2015-08-04 16:58:00  anaheim
1      723 2015-08-04 16:52:00  anaheim
2      724 2015-08-04 16:52:00  anaheim
3      725 2015-08-04 16:50:00  anaheim
4      726 2015-08-04 16:49:00  anaheim


In [23]:
#import wait time data
cursor.execute("SELECT * FROM recorded_ca_crowd_data")
wtrows = cursor.fetchall()
#print wtrows
waitdf = pd.DataFrame( [[ij for ij in i] for i in wtrows] )
print waitdf.head()

                   0              1   2   3     4   5   6   7   8   9  ...  \
0 2015-08-04 03:54:39  1438685370401  -5   0  68.5   0   0   0   0   0 ...   
1 2015-08-04 03:54:39  1438685370404  -5   0  68.5   0   0   0   0   0 ...   
2 2015-08-04 03:54:39  1438685679566  -5   0  68.5   0   0   0   0   0 ...   
3 2015-08-04 03:54:39  1438685679566  -5   0  68.5   0   0   0   0   0 ...   
4 2015-08-04 03:54:39  1438685679566  -5   0  68.5   0   0   0   0   0 ...   

   17  18  19  20  21  22  23  24  25  26  
0   0   0   0   0   0   0   0   0   0   0  
1   0   0   0   0   0   0   0   0   0   0  
2   0   0   0   0   0   0   0   0   0   0  
3   0   0   0   0   0   0   0   0   0   0  
4   0   0   0   0   0   0   0   0   0   0  

[5 rows x 27 columns]


In [24]:
alltimes = waitdf.ix[:,5:34]
alltimes2 = alltimes.apply(lambda x: pd.to_numeric(x, errors='coerce'))
#print alltimes2.ix[:,5]
meanwaits = pd.DataFrame(alltimes2.apply(np.nanmean, axis=1))
meanwaits.rename(columns={0: 'meanwait'}, inplace=True)
meanwaits.head()
#nb: this runs an error but it's ok, it's the mean of empty groups

,meanwait
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [25]:
frames = [waitdf.ix[:,0], meanwaits]
dl_data = pd.concat(frames, axis=1)
dl_data = dl_data.dropna()
dl_data.rename(columns={0: 'timestamp'}, inplace=True)
print dl_data.head()
#dl_data.plot(x='timestamp', y='meanwait')

            timestamp  meanwait
0 2015-08-04 03:54:39       0.0
1 2015-08-04 03:54:39       0.0
2 2015-08-04 03:54:39       0.0
3 2015-08-04 03:54:39       0.0
4 2015-08-04 03:54:39       0.0


In [26]:
#bin wait time data by hour
dl_data['hod'] = [datetime.datetime(dt.year, dt.month, dt.day, dt.hour) for dt in dl_data.timestamp]
dl_meanwait_by_hour = pd.DataFrame(dl_data.groupby(['hod'])['meanwait'].mean()).reset_index('hod')
dl_meanwait_by_hour.set_index=dl_meanwait_by_hour['hod']
dl_meanwait_by_hour.head()

,hod,meanwait
0,2015-08-04 03:00:00,0.000000
1,2015-08-04 04:00:00,0.000000
2,2015-08-04 08:00:00,0.000000
3,2015-08-04 14:00:00,34.318182
4,2015-08-04 15:00:00,34.431818


In [27]:
#now bin tweets by hour
tweetdf['hod'] = [datetime.datetime(dt.year, dt.month, dt.day, dt.hour) for dt in tweetdf.timestamps]
tweets_per_hour = pd.DataFrame(tweetdf.groupby(['hod'])['tweetid'].count()).reset_index()
tweets_per_hour.head()

,hod,tweetid
0,2015-08-04 00:00:00,9
1,2015-08-04 01:00:00,9
2,2015-08-04 04:00:00,2
3,2015-08-04 07:00:00,3
4,2015-08-04 08:00:00,11


In [28]:
#join tables
tweetwaits = pd.merge(dl_meanwait_by_hour, tweets_per_hour, on='hod')
tweetwaits['hour'] = [dt.hour for dt in tweetwaits.hod]
tweetwaits.head()

,hod,meanwait,tweetid,hour
0,2015-08-04 04:00:00,0.000000,2,4
1,2015-08-04 08:00:00,0.000000,11,8
2,2015-08-04 14:00:00,34.318182,12,14
3,2015-08-04 15:00:00,34.431818,13,15
4,2015-08-04 16:00:00,34.772727,17,16


In [32]:
#run a test correlation meanwait and tweets?
#cortest = scipy.stats.pearsonr(tweetwaits.meanwait, tweetwaits.tweetid)
cortest = scipy.stats.pearsonr(np.log(tweetwaits.meanwait+1), np.log(tweetwaits.tweetid + 1))
cortest #not great but significant

(0.11105832259363045, 0.023157745055188538)

In [33]:
#import weather data
cursor.execute("SELECT TemperatureF, Wind_SpeedMPH, PrecipitationIN, Conditions, DateUTC FROM anaheim_weather WHERE DateUTC > '2015-08-03' AND DateUTC < '2015-09-13' ")
wrows = cursor.fetchall()
weatherdf = pd.DataFrame( [[ij for ij in i] for i in wrows] )
weatherdf.rename(columns={0: 'temp', 1:'wind', 2:'precip', 3:'conditions', 4:'datetimeUTC'}, inplace=True)

In [34]:
#fix time zone on weather data
weatherdf['datetimeUTC'] =  pd.to_datetime(weatherdf['datetimeUTC'], format='%Y-%m-%d %H:%M:%S.')
weatherdf['timestamp'] = weatherdf['datetimeUTC'] - datetime.timedelta(hours=8)
weatherdf['hod'] = [datetime.datetime(dt.year, dt.month, dt.day, dt.hour) for dt in weatherdf.timestamp] 
weatherdf.head()

,temp,wind,precip,conditions,datetimeUTC,timestamp,hod
0,81.0,6.9,N/A,Clear,2015-08-03 00:53:00,2015-08-02 16:53:00,2015-08-02 16:00:00
1,79.0,4.6,N/A,Clear,2015-08-03 01:53:00,2015-08-02 17:53:00,2015-08-02 17:00:00
2,77.0,5.8,N/A,Clear,2015-08-03 02:53:00,2015-08-02 18:53:00,2015-08-02 18:00:00
3,75.0,Calm,N/A,Clear,2015-08-03 03:53:00,2015-08-02 19:53:00,2015-08-02 19:00:00
4,73.0,3.5,N/A,Clear,2015-08-03 04:53:00,2015-08-02 20:53:00,2015-08-02 20:00:00


In [35]:
#bin weather data
weatherdf['wind']= weatherdf['wind'].astype(str)
weatherdf.loc[weatherdf.wind == 'Calm'] = 0
weatherdf['wind'] = pd.to_numeric(weatherdf['wind'])
weatherdf.loc[weatherdf.wind < -100] = float('NaN')
hourly_weather = pd.DataFrame(weatherdf.groupby(['hod']).agg({'temp': np.nanmean, 'wind': np.nanmean, 'conditions': 'first'})).reset_index()
hourly_weather.head()

,hod,conditions,wind,temp
0,1970-01-01 00:00:00,0,0.0,0.0
1,2015-08-02 16:00:00,Clear,6.9,81.0
2,2015-08-02 17:00:00,Clear,4.6,79.0
3,2015-08-02 18:00:00,Clear,5.8,77.0
4,2015-08-02 20:00:00,Clear,3.5,73.0


In [36]:
#join to existing table
tweetwaits = pd.merge(tweetwaits, hourly_weather, on='hod')
#tweetwaits['hour'] = [dt.hour for dt in tweetwaits.hod]

In [37]:
#get business days and holidays
#tweetwaits.info() #get info on the dataframe
tweetwaits['business_day'] = [dt.weekday() >= 5 for dt in tweetwaits.hod]
us_holidays = holidays.UnitedStates()
tweetwaits['holiday'] = [day in us_holidays for day in tweetwaits.hod]

In [38]:
myfunc = lambda tweetwaits: True if ((tweetwaits['holiday']) or (tweetwaits['business_day'])) else False
tweetwaits['we_ho'] = tweetwaits.apply(myfunc, axis=1)

In [40]:
#preview data
tweetwaits

,hod,meanwait,tweetid,hour,conditions,wind,temp,business_day,holiday,we_ho
0,2015-08-04 14:00:00,34.318182,12,14,Clear,6.900000,89.100000,False,False,False
1,2015-08-04 15:00:00,34.431818,13,15,Clear,5.800000,89.100000,False,False,False
2,2015-08-04 16:00:00,34.772727,17,16,Clear,8.100000,87.100000,False,False,False
3,2015-08-04 17:00:00,33.920455,8,17,Clear,6.900000,82.000000,False,False,False
4,2015-08-04 18:00:00,33.409091,13,18,Clear,4.600000,78.100000,False,False,False
5,2015-08-04 19:00:00,31.818182,21,19,Clear,4.600000,75.900000,False,False,False
6,2015-08-04 21:00:00,28.068182,14,21,Clear,4.600000,73.900000,False,False,False
7,2015-08-05 08:00:00,6.647727,2,8,Clear,3.500000,80.100000,False,False,False
8,2015-08-05 09:00:00,25.568182,8,9,Clear,5.800000,84.900000,False,False,False
9,2015-08-05 10:00:00,29.090909,13,10,Clear,6.900000,86.000000,False,False,False


In [41]:
#export data to new database table so don't need to constantly rerun for testing
#cursor.execute("DROP TABLE IF EXISTS dl_test_cleaned"
tweetwaits.to_sql(name='ca_test_clean', con=conn, flavor='mysql', schema=None, if_exists='replace', index=True)

In [42]:
#close SQL connection
conn.close()

In [ ]:
#some EDA plots (Disneyland)
dl_df = tweetwaits
sns.set_style("whitegrid")
sns.plt.figure(1)

sns.plt.subplot(411)
#note log transforms
sns.plt.plot(dl_df['temp'], np.log(dl_df['meanwait']+1), 'o')

sns.plt.subplot(412)
sns.plt.plot(dl_df['wind'], np.log(dl_df['meanwait']+1), 'o', color='forestgreen')

sns.plt.subplot(413)
sns.plt.plot(np.log(dl_df['tweetid']+1), np.log(dl_df['meanwait']+1), 'o', color='darkred')

sns.plt.subplot(414)
sns.plt.plot(dl_df['hour'], np.log(dl_df['meanwait']+1), 'o', color='purple')
sns.plt.show()